First try of correlating albedo with DEM

In [1]:
import ee
import geemap
# from geemap import cartoee
# import cartopy.crs as ccrs
# # import geemap.colormaps as cm
import matplotlib.pyplot as plt
# # from geetools import batch
# import numpy as np
# import pandas as pd
# import seaborn as sns
# import time 

In [2]:
Map = geemap.Map()
Map 

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

# Prepare aoi and ice mask

In [4]:
# aoi = ee.Image('OSU/GIMP/2000_ICE_OCEAN_MASK').geometry().bounds() # this is too large, discarded
# aoi = ee.Geometry.Polygon(
#     [[[-36.29516924635421, 83.70737243835941],
#     [-51.85180987135421, 82.75597137647488],
#     [-61.43188799635421, 81.99879137488564],
#     [-74.08813799635422, 78.10103528196419],
#     [-70.13305987135422, 75.65372336709613],
#     [-61.08032549635421, 75.71891096312955],
#     [-52.20337237135421, 60.9795530382023],
#     [-43.41430987135421, 58.59235996703347],
#     [-38.49243487135421, 64.70478286561182],
#     [-19.771731746354217, 69.72271161037442],
#     [-15.728762996354217, 76.0828635948066],
#     [-15.904544246354217, 79.45091003031243],
#     [-10.015872371354217, 81.62328742628017],
#     [-26.627200496354217, 83.43179828852398],
#     [-31.636966121354217, 83.7553561747887]]]
# ) # whole greenland
# aoi = ee.Geometry.Polygon(
#     [[[-75.35327725640606, 78.15797707936824],
#       [-58.137306661848434, 69.59945512283268],
#       [-51.82415036596651, 59.897134149764156],
#       [-42.233465551083604, 59.260337764670496],
#       [-61.95501079278244, 79.65995314962508]]]
# ) # western greenland
# aoi = ee.Geometry.Polygon(
#     [[[-57.19932402786638, 71.06394313461017],
#       [-57.19932402786638, 63.62039597689339],
#       [-44.45518340286638, 63.62039597689339],
#       [-44.45518340286638, 71.06394313461017]]]
# ) # darkzone
# aoi = ee.Geometry.Polygon(
#         [[[-51.26670684036638, 71.10667904834996],
#           [-50.613642172036386, 69.35172505065574],
#           [-51.925886527866396, 67.4097894479071],
#           [-51.20992413915373, 66.27518798747208],
#           [-51.86713118464301, 66.04083709002118],
#           [-51.68516540592766, 65.85920268639616],
#           [-50.14610137161638, 63.77010325395928],
#           [-44.45518340286638, 63.62039597689339],
#           [-44.45518340286638, 71.06394313461017]]]) # only connected ice
# Map.addLayer(aoi, {}, 'aoi')

'''
how to get corner coordinates: 
https://gis.stackexchange.com/questions/318959/get-lon-lat-of-a-top-left-corner-for-geometry-in-google-earth-engine
'''
# listCoords = ee.Array.cat(aoi.coordinates(), 1); 

# get the X-coordinates
# xCoords = listCoords.slice(1, 0, 1)
# yCoords = listCoords.slice(1, 1, 2)

# xMin = xCoords.reduce('min', [0]).get([0,0]).getInfo()
# xMax = xCoords.reduce('max', [0]).get([0,0]).getInfo()
# yMin = yCoords.reduce('min', [0]).get([0,0]).getInfo()
# yMax = yCoords.reduce('max', [0]).get([0,0]).getInfo()

'''
alternatively, https://gist.github.com/graydon/11198540
list of country-bounding-boxes
'GL': ('Greenland', (-73.297, 60.03676, -12.20855, 83.64513)),
'''
# xMin = -73.297
# xMax = -12.20855
# yMin = 60.03676
# yMax = 83.64513


"\nalternatively, https://gist.github.com/graydon/11198540\nlist of country-bounding-boxes\n'GL': ('Greenland', (-73.297, 60.03676, -12.20855, 83.64513)),\n"

In [4]:
greenlandmask = ee.Image('OSU/GIMP/2000_ICE_OCEAN_MASK') \
                   .select('ice_mask').eq(1) #'ice_mask', 'ocean_mask'
# arcticDEM = ee.ImageCollection('UMN/PGC/ArcticDEM/V3/2m').select('elevation').updateMask(greenlandmask) # strip
arcticDEM = ee.Image("UMN/PGC/ArcticDEM/V3/2m_mosaic").select('elevation').updateMask(greenlandmask) # mosaic

# arcticDEMgreenland = arcticDEM.updateMask(greenlandmask)

visPara = {'min': 0,  'max': 3000.0, 'palette': ['0d13d8', '60e1ff', 'ffffff']}


Map.addLayer(arcticDEM, visPara, 'Arctic DEM terrain')
Map.setCenter(-41.0, 74.0, 3)
# Map.add_colorbar(visPara, label="Elevation (m)", discrete=False, orientation="vertical", layer_name="Arctic DEM terrain")

# Prepare Harmonized Albedo

In [5]:
date_start = ee.Date.fromYMD(1984, 1, 1)
date_end = ee.Date.fromYMD(2020, 12, 31)


# yearnum = 2020

# date_start = ee.Date.fromYMD(yearnum, 1, 1)
# date_end = ee.Date.fromYMD(yearnum, 12, 1)
# titlestr = "albedo " + str(yearnum)# + "-" + str(yearnum+4)

In [6]:
def addTotalAlbedo(image):
    albedo = image.expression(
        '1.0065 * Blue + 1.1902 * Green - 2.6738 * Red + 1.1700 * NIR + 0.9211 * SWIR1 - 1.1667 * SWIR2 + 0.1757',
        {
            'Blue': image.select('Blue'),
            'Green': image.select('Green'),
            'Red': image.select('Red'),
            'NIR': image.select('NIR'),
            'SWIR1': image.select('SWIR1'),
            'SWIR2': image.select('SWIR2')
        }
    ).rename('totalAlbedo')
    return image.addBands(albedo).copyProperties(image, ['system:time_start'])

def addVisnirAlbedo(image):
    albedo = image.expression(
        '0.7605 * Blue + 0.8090 * Green - 1.8376 * Red + 0.9145 * NIR + 0.1627',
        {
            'Blue': image.select('Blue'),
            'Green': image.select('Green'),
            'Red': image.select('Red'),
            'NIR': image.select('NIR')
        }
    ).rename('visnirAlbedo')
    return image.addBands(albedo).copyProperties(image, ['system:time_start'])


## If VIS-NIR bands albedo

In [7]:
''''if viz-nir bands albedo'''
rmaCoefficients = {
  'itcpsL7': ee.Image.constant([-0.0084, -0.0065, 0.0022, -0.0768]),
  'slopesL7': ee.Image.constant([1.1017, 1.0840, 1.0610, 1.2100]),
  'itcpsS2': ee.Image.constant([0.0210, 0.0167, 0.0155, -0.0693]),
  'slopesS2': ee.Image.constant([1.0849, 1.0590, 1.0759, 1.1583])
}; #rma
# Function to get and rename bands of interest from OLI.
def renameOli(img):
  return img.select(
    ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'QA_PIXEL', 'QA_RADSAT'], #'QA_PIXEL', 'QA_RADSAT'
    ['Blue',  'Green', 'Red',   'NIR',   'QA_PIXEL', 'QA_RADSAT']) #'QA_PIXEL', 'QA_RADSAT'

# Function to get and rename bands of interest from ETM+, TM.
def renameEtm(img):
  return img.select(
    ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'QA_PIXEL', 'QA_RADSAT'], #,   'QA_PIXEL', 'QA_RADSAT'
    ['Blue',  'Green', 'Red',   'NIR',   'QA_PIXEL', 'QA_RADSAT']) #, 'QA_PIXEL', 'QA_RADSAT'

# Function to get and rename bands of interest from Sentinel 2.
def renameS2(img):
  return img.select(
    ['B2',   'B3',    'B4',  'B8',  'QA60', 'SCL'],
    ['Blue', 'Green', 'Red', 'NIR', 'QA60', 'SCL']
  )

def oli2oli(img):
  return img.select(['Blue', 'Green', 'Red', 'NIR']) \
    .toFloat()

def etm2oli(img):
  return img.select(['Blue', 'Green', 'Red', 'NIR']) \
    .multiply(rmaCoefficients["slopesL7"]) \
    .add(rmaCoefficients["itcpsL7"]) \
    .toFloat()
    # .round() \
    # .toShort() 
    # .addBands(img.select('pixel_qa'))

def s22oli(img):
  return img.select(['Blue', 'Green', 'Red', 'NIR']) \
    .multiply(rmaCoefficients["slopesS2"]) \
    .add(rmaCoefficients["itcpsS2"]) \
    .toFloat()
    # .round() \
    # .toShort() # convert to Int16
    # .addBands(img.select('pixel_qa'))

def imRangeFilter(image):
  maskMax = image.lt(1)
  maskMin = image.gt(0)
  return image.updateMask(maskMax).updateMask(maskMin)

'''
Cloud mask for Landsat data based on fmask (QA_PIXEL) and saturation mask 
based on QA_RADSAT.
Cloud mask and saturation mask by sen2cor.
Codes provided by GEE official. '''

# the Landsat 8 Collection 2
def maskL8sr(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Cirrus
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
  qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)

  # Apply the scaling factors to the appropriate bands.
  # opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  # thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

  # Replace the original bands with the scaled ones and apply the masks.
  #image.addBands(opticalBands, {}, True) \ maybe not available in python api
  return image.select(['Blue', 'Green', 'Red', 'NIR']).multiply(0.0000275).add(-0.2) \
    .updateMask(qaMask) \
    .updateMask(saturationMask)

  
# the Landsat 4, 5, 7 Collection 2
def maskL457sr(image):
  # Bit 0 - Fill
  # Bit 1 - Dilated Cloud
  # Bit 2 - Unused
  # Bit 3 - Cloud
  # Bit 4 - Cloud Shadow
  qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
  saturationMask = image.select('QA_RADSAT').eq(0)

  # Apply the scaling factors to the appropriate bands.
  # opticalBands = image.select('SR_B.')
  # opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  # thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

  # Replace the original bands with the scaled ones and apply the masks.
  return image.select(['Blue', 'Green', 'Red', 'NIR']).multiply(0.0000275).add(-0.2) \
      .updateMask(qaMask) \
      .updateMask(saturationMask)
 #
 # Function to mask clouds using the Sentinel-2 QA band
 # @param {ee.Image} image Sentinel-2 image
 # @return {ee.Image} cloud masked Sentinel-2 image
 #
def maskS2sr(image):
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloudBitMask = 1 << 10
  cirrusBitMask = 1 << 11
  # Bits 1 is saturated or defective pixel
  not_saturated = image.select('SCL').neq(1)
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
      .And(qa.bitwiseAnd(cirrusBitMask).eq(0)) 

  return image.updateMask(mask).updateMask(not_saturated)


In [8]:
# Define function to prepare OLI images.
def prepOli(img):
  orig = img
  img = renameOli(img)
  img = maskL8sr(img)
  img = oli2oli(img)
  # img = addTotalAlbedo(img)
  img = addVisnirAlbedo(img)
  return ee.Image(img.copyProperties(orig, orig.propertyNames()))

# Define function to prepare ETM+/TM images.
def prepEtm(img):
  orig = img
  img = renameEtm(img)
  img = maskL457sr(img)
  img = etm2oli(img)
  # img = addTotalAlbedo(img)
  img = addVisnirAlbedo(img)
  return ee.Image(img.copyProperties(orig, orig.propertyNames()))

# Define function to prepare S2 images.
def prepS2(img):
  orig = img
  img = renameS2(img)
  img = maskS2sr(img)
  img = s22oli(img)
  # img = addTotalAlbedo(img)
  img = addVisnirAlbedo(img)
  return ee.Image(img.copyProperties(orig, orig.propertyNames()).set('SATELLITE', 'SENTINEL_2'))


In [9]:
# create filter for image collection
colFilter = ee.Filter.And(
    ee.Filter.geometry(aoi), # filterbounds not available on python api https://github.com/google/earthengine-api/issues/83
    ee.Filter.date(date_start, date_end),
    ee.Filter.calendarRange(6, 8, 'month'),
    # ee.Filter.lt('CLOUD_COVER', 50)
)

s2colFilter =  ee.Filter.And(
    ee.Filter.geometry(aoi), # filterbounds not available on python api https://github.com/google/earthengine-api/issues/83
    ee.Filter.date(date_start, date_end),
    ee.Filter.calendarRange(6, 8, 'month'),
    ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 50)
)

oliCol = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
            .filter(colFilter) \
            .map(prepOli) \
            .select(['visnirAlbedo']) # .select(['totalAlbedo']) or  .select(['viznirAlbedo'])
etmCol = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
            .filter(colFilter) \
            .map(prepEtm) \
            .select(['visnirAlbedo']) # .select(['totalAlbedo']) or  .select(['viznirAlbedo'])
tmCol = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
            .filter(colFilter) \
            .map(prepEtm) \
            .select(['visnirAlbedo']) # .select(['totalAlbedo']) or  .select(['viznirAlbedo'])
tm4Col = ee.ImageCollection('LANDSAT/LT04/C02/T1_L2') \
            .filter(colFilter) \
            .map(prepEtm) \
            .select(['visnirAlbedo']) # .select(['totalAlbedo']) or  .select(['viznirAlbedo'])
s2Col = ee.ImageCollection('COPERNICUS/S2_SR') \
            .filter(s2colFilter) \
            .map(prepS2) \
            .select(['visnirAlbedo']) # .select(['totalAlbedo']) or  .select(['viznirAlbedo'])
landsatCol = oliCol.merge(etmCol).merge(tmCol).merge(tm4Col)
multiSat = landsatCol.merge(s2Col).map(imRangeFilter).sort('system:time_start', True)
